In [113]:
import pandas as pd
data = pd.read_fwf("D:\Drive UPTC\phd\deployment proyect\Data\IDEAM_2022\RAD_20229050172462\PT_2_MX_D@35160000.data")

In [114]:
data

,Fecha|Valor
0,2017-01-28 07:00:00|0.0
1,2017-01-29 07:00:00|0.0
2,2017-01-30 07:00:00|0.0
3,2017-01-31 07:00:00|0.0
4,2017-02-01 07:00:00|0.0
...,...
987,2019-10-12 07:00:00|0.0
988,2019-10-13 07:00:00|0.0
989,2019-10-14 07:00:00|0.0
990,2019-10-15 07:00:00|0.0


In [115]:
import pandas as pd
# Read Excel file
df = pd.read_excel('D:\Temp\IDEAM_2022\CNE_IDEAM.xls')
df.head()

,OBJECTID,CODIGO,nombre,CATEGORIA,TECNOLOGIA,ESTADO,FECHA_INSTALACION,altitud,latitud,longitud,...,MUNICIPIO,AREA_OPERATIVA,AREA_HIDROGRAFICA,ZONA_HIDROGRAFICA,observacion,CORRIENTE,FECHA_SUSPENSION,SUBZONA_HIDROGRAFICA,ENTIDAD,subred
0,1,52057100,RUMICHACA - AUT [52057100],Limnigráfica,Automática con Telemetría,Activa,2016-11-15,2582,0.813786,-77.661978,...,Ipiales,Area Operativa 07 - Nariño-Putumayo,Pacifico,Patía,NaN,Guaitara,NaT,Río Guáitara,INSTITUTO DE HIDROLOGIA METEOROLOGIA Y ESTUDIO...,NaN
1,2,52055170,LA JOSEFINA - AUT [52055170],Climática Principal,Automática con Telemetría,Activa,2005-12-10,2450,0.930306,-77.491194,...,Contadero,Area Operativa 07 - Nariño-Putumayo,Pacifico,Patía,NaN,Guaitara,NaT,Río Guáitara,INSTITUTO DE HIDROLOGIA METEOROLOGIA Y ESTUDIO...,"Radiación Ultravioleta,Radiación Visible,"
2,3,52055220,EL PARAISO - AUT [52055220],Climática Principal,Automática con Telemetría,Activa,2004-03-01,3120,1.070611,-77.636889,...,Túquerres,Area Operativa 07 - Nariño-Putumayo,Pacifico,Patía,NaN,Guaitara,NaT,Río Guáitara,INSTITUTO DE HIDROLOGIA METEOROLOGIA Y ESTUDIO...,"Altiplano Nariñense-Pacifico,Radiación Ultravi..."
3,4,44015070,EL PEPINO - AUT [44015070],Climática Principal,Automática con Telemetría,Activa,2005-11-11,760,1.082889,-76.667111,...,Mocoa,Area Operativa 07 - Nariño-Putumayo,Amazonas,Caquetá,NaN,Guaitara,NaT,Alto Caqueta,INSTITUTO DE HIDROLOGIA METEOROLOGIA Y ESTUDIO...,"Radiación Ultravioleta,Radiación Visible,"
4,5,48015040,PUERTO NARINO - AUT [48015040],Climática Principal,Automática con Telemetría,Activa,2005-07-19,158,-3.780306,-70.362639,...,Puerto Nariño,Area Operativa 11 - Cundinamarca-Amazonas-San ...,Amazonas,Amazonas - Directos,NaN,Amazonas,NaT,Directos Río Amazonas (mi),INSTITUTO DE HIDROLOGIA METEOROLOGIA Y ESTUDIO...,"Radiación Ultravioleta,Radiación Visible,"


In [116]:
df_boyaca = df[df['DEPARTAMENTO'] == 'Boyacá']
df_boyaca

,OBJECTID,CODIGO,nombre,CATEGORIA,TECNOLOGIA,ESTADO,FECHA_INSTALACION,altitud,latitud,longitud,...,MUNICIPIO,AREA_OPERATIVA,AREA_HIDROGRAFICA,ZONA_HIDROGRAFICA,observacion,CORRIENTE,FECHA_SUSPENSION,SUBZONA_HIDROGRAFICA,ENTIDAD,subred
61,62,24035380,NEVADO DEL COCUY - AUT [24035380],Climática Principal,Automática con Telemetría,Activa,2005-11-18 19:00:00,4676,6.509111,-72.314167,...,Guicán,Area Operativa 06 - Boyacá-Casanare-Vichada,Magdalena Cauca,Sogamoso,Solicitud interna,0,NaT,Río Chicamocha,INSTITUTO DE HIDROLOGIA METEOROLOGIA Y ESTUDIO...,NaN
62,63,24037001,"LA PLAYA, CARDENILLO [24037001]",Limnigráfica,Automática sin Telemetría,Activa,2013-10-23 19:00:00,3742,6.499556,-72.376139,...,Guicán,Area Operativa 06 - Boyacá-Casanare-Vichada,Magdalena Cauca,Sogamoso,Requerimiento interno,Rio Cardenillo,NaT,Río Chicamocha,INSTITUTO DE HIDROLOGIA METEOROLOGIA Y ESTUDIO...,"Altiplano Cundiboyacense y oriente de Santander,"
106,107,35085070,SANTA MARIA - AUT [35085070],Climática Principal,Automática con Telemetría,Activa,2005-03-10 00:00:00,1300,4.841250,-73.256694,...,Santa María (Boyacá),Area Operativa 06 - Boyacá-Casanare-Vichada,Orinoco,Meta,NaN,Lagunilla,NaT,Río Garagoa,INSTITUTO DE HIDROLOGIA METEOROLOGIA Y ESTUDIO...,"Radiación Ultravioleta,Radiación Visible,"
117,118,35075080,PARAMO RABANAL - AUT [35075080],Climática Principal,Automática con Telemetría,Activa,2005-03-13 00:00:00,3398,5.392389,-73.562778,...,Ventaquemada,Area Operativa 06 - Boyacá-Casanare-Vichada,Orinoco,Meta,NaN,Bogota,NaT,Río Garagoa,INSTITUTO DE HIDROLOGIA METEOROLOGIA Y ESTUDIO...,NaN
121,122,35075070,CHINAVITA - AUT [35075070],Agrometeorológica,Automática con Telemetría,Activa,2005-03-06 00:00:00,212,5.219250,-73.350389,...,Chinavita,Area Operativa 06 - Boyacá-Casanare-Vichada,Orinoco,Meta,NaN,Guali,NaT,Río Garagoa,INSTITUTO DE HIDROLOGIA METEOROLOGIA Y ESTUDIO...,"Altiplano Cundiboyacense,Radiación Ultraviolet..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4441,4442,35077120,CARACOL EL [35077120],Limnigráfica,Convencional,Activa,1974-12-15 00:00:00,1298,5.056861,-73.390333,...,Garagoa,Area Operativa 06 - Boyacá-Casanare-Vichada,Orinoco,Meta,20/05/2022$Migracion$REQ 2022-003579. El Coord...,Garagoa,NaT,Río Garagoa,INSTITUTO DE HIDROLOGIA METEOROLOGIA Y ESTUDIO...,"Altiplano Cundiboyacense y oriente de Santander,"
4443,4444,24035350,ANDALUCIA [24035350],Climática Principal,Convencional,Activa,1996-09-15 00:00:00,3265,5.901139,-73.058333,...,Duitama,Area Operativa 06 - Boyacá-Casanare-Vichada,Magdalena Cauca,Sogamoso,20/05/2022$Migracion$REQ 2022-003579. El Coord...,0,NaT,Río Chicamocha,INSTITUTO DE HIDROLOGIA METEOROLOGIA Y ESTUDIO...,"Altiplano Cundiboyacense y oriente de Santander,"
4445,4446,35077090,PUENTE ADRIANA [35077090],Limnimétrica,Convencional,Activa,1985-08-15 00:00:00,258,5.385000,-73.361639,...,Jenesano,Area Operativa 06 - Boyacá-Casanare-Vichada,Orinoco,Meta,20/05/2022$Migracion$REQ 2022-003579. El Coord...,Jenesano,NaT,Río Garagoa,INSTITUTO DE HIDROLOGIA METEOROLOGIA Y ESTUDIO...,"RED ALERTAS - AREA OPERATIVA 06,"
4447,4448,24030840,ANTENA TV RUSIA LA [24030840],Pluviométrica,Convencional,Activa,1980-08-15 00:00:00,3650,5.891639,-73.082389,...,Duitama,Area Operativa 06 - Boyacá-Casanare-Vichada,Magdalena Cauca,Sogamoso,20/05/2022$Migracion$REQ 2022-003579. El coord...,0,NaT,Río Suárez,INSTITUTO DE HIDROLOGIA METEOROLOGIA Y ESTUDIO...,"RED ALERTAS - AREA OPERATIVA 06,"


In [117]:
# PT_10_TT_D: Precipitación total diaria, filtrado por esta etiqueta en los archivos y por las estaciones de Boyacá

import os

nombre_carpeta = 'D:\Temp\IDEAM_2022\RAD_20229050172462'

contenido = os.listdir(nombre_carpeta)

files = []
for elemento in contenido:
    if elemento != 'desktop.ini' and 'PT_10_TT_D' in elemento:
        print(os.path.join(elemento))
        files.append(elemento)

PT_10_TT_D@23125160.data
PT_10_TT_D@24035360.data
PT_10_TT_D@24035370.data
PT_10_TT_D@24035410.data
PT_10_TT_D@24035430.data
PT_10_TT_D@35075070.data
PT_10_TT_D@35075080.data
PT_10_TT_D@35085060.data
PT_10_TT_D@35085070.data
PT_10_TT_D@35085080.data
PT_10_TT_D@35095120.data
PT_10_TT_D@35160000.data
PT_10_TT_D@35160010.data
PT_10_TT_D@35160020.data
PT_10_TT_D@35165000.data
PT_10_TT_D@35167000.data
PT_10_TT_D@35167010.data
PT_10_TT_D@35167020.data
PT_10_TT_D@35195060.data
PT_10_TT_D@35215030.data
PT_10_TT_D@35217090.data
